# Access CESM2-LE data

In [1]:
# import packages
import intake
import xarray as xr
import numpy as np
import cupy as cp
import cupy_xarray
# import matplotlib.pyplot as plt

def degcos(x):
    return np.cos(x / 180 * np.pi)

UBILEXSCRATCH = "/storage/scratch/users/hb22g102"

In [ ]:
catalog = intake.open_esm_datastore("https://ncar-cesm2-lens.s3-us-west-2.amazonaws.com/catalogs/aws-cesm2-le.json")
catalog_subset = catalog.search(frequency = "daily", variable =["U", "V"], forcing_variant = "cmip6", experiment="historical")
dsets = catalog_subset.to_dataset_dict(storage_options={'anon': True}, xarray_open_kwargs={"engine": "kvikio"}) # , xarray_open_kwargs={"engine": "kvikio"}
dsets.keys()
# historical_cmip6 = dsets['atm.historical.daily.cmip6']
# future_cmip6 = dsets['atm.ssp370.daily.cmip6']
# ds = xr.concat([historical_cmip6, future_cmip6], dim='time')
# ds = ds.isel(member_id=np.delete(np.arange(50), np.arange(0, 50, 5))) # dont want the "first ten"
# ds = ds.isel(lon=np.append(np.arange(10), np.arange(230, len(ds.lon)))).isel(lat=np.arange(107, 182)) # na sector
# ods = ds.copy()

In [26]:
dsets['atm.historical.daily.cmip6'].isel(member_id=0, time=0, lev=0, lat=0).to_zarr(f"{UBILEXSCRATCH}/test.zarr")

In [ ]:
xr.open_dataset(f"{UBILEXSCRATCH}/test.zarr", engine="kvikio", consolidated=False)

In [4]:
future_cmip6 = dsets['atm.ssp370.daily.cmip6']

In [9]:
type(future_cmip6["U"].isel(member_id=0, time=0, lev=0, lat=0).load().data)

numpy.ndarray

In [ ]:
ds = ds.roll(lon=144, roll_coords=True)
a = np.array(ds["lon"])
a[:144] = - 360 + a[:144]
ds["lon2"] = xr.DataArray(a, coords={"lon": ds["lon"]})

In [ ]:
ds = ds.swap_dims({"lon": "lon2"})
ds = ds.drop("lon")
ds = ds.rename({"lon2": "lon"})
ds = ds.isel(lon=np.arange(90, 160)).isel(lat=np.arange(107, 182))

In [ ]:
a = 6.371e6
newds = ds.isel(lev=25, time=0, member_id=0)
newds["vort"] = (newds["V"].differentiate("lon") - (newds["U"] * degcos(newds["lat"])).differentiate("lat")) / a / degcos(newds["lat"])
newds["vort"].attrs["long_name"] = "vorticity"
newds["vort"].attrs["units"] = "1/s"

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,8))
newds["vort"].plot(ax=ax)

In [ ]:
ds["U"]